# Faster autoscaling on Amazon SageMaker realtime endpoints (Step Scaling)

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)


---

In this notebook we show how the new faster autoscaling feature helps scale sagemaker inference endpoints by almost 6x faster than earlier.

We deploy Meta's `Llama3-8B-Instruct` model to an Amazon SageMaker realtime endpoint using Text Generation Inference (TGI) Deep Learning Container (DLC) and apply <span style='color:green'><b>Step Scaling</b></span> autoscaling policies to the endpoint.


<span class="alert alert-block alert-warning">Please use at least <strong>`m5.2xlarge`</strong> or larger instance types if running this on Amazon SageMaker Notebook Instance.</span>


## Prerequisites

<div style="border: 1px solid #f00; border-radius: 5px; padding: 10px; background-color: #fee;">
Before using this notebook please ensure you have access to an active access token from HuggingFace and have accepted the license agreement from Meta.

- **Step 1:** Create user access token in HuggingFace (HF). Refer [here](https://huggingface.co/docs/hub/security-tokens) on how to create HF tokens.
- **Step 2:** Login to [HuggingFace](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/tree/main) and navigate to *Meta-Llama-3-8B-Instruct** home page.
- **Step 3:** Accept META LLAMA 3 COMMUNITY LICENSE AGREEMENT by following the instructions [here](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/tree/main)
- **Step 4:** Wait for the approval email from META (Approval may take any where b/w 1-3 hrs)
</div>

Ensure python version of kernel is 3.10

In [1]:
!python --version

Python 3.10.14


Install packages using uv, an extremely fast python package installer. Read more about uv here <https://astral.sh/blog/uv>

In [2]:
!pip install uv && uv pip install -U ipywidgets
!uv pip install -r requirements.txt

Resolved 22 packages in 44ms                                                 
Uninstalled 5 packages in 13ms
Installed 5 packages in 8ms6.0                                      
 - exceptiongroup==1.2.1
 + exceptiongroup==1.2.2
 - ipython==8.24.0
 + ipython==8.26.0
 - prompt-toolkit==3.0.45
 + prompt-toolkit==3.0.47
 - pure-eval==0.2.2
 + pure-eval==0.2.3
 - typing-extensions==4.12.0
 + typing-extensions==4.12.2
Resolved 65 packages in 205ms3.12.1                                          
Uninstalled 5 packages in 7ms
Installed 5 packages in 7ms4.0                                      
 - exceptiongroup==1.2.2
 + exceptiongroup==1.2.1
 - ipython==8.26.0
 + ipython==8.24.0
 - prompt-toolkit==3.0.47
 + prompt-toolkit==3.0.45
 - pure-eval==0.2.3
 + pure-eval==0.2.2
 - typing-extensions==4.12.2
 + typing-extensions==4.12.0


Restart kernel after installing packages

In [3]:
# restart kernel
from IPython.core.display import HTML

HTML("<script>Jupyter.notebook.kernel.restart()</script>")
print("Kernel restarted successfully!")

Kernel restarted successfully!


In [4]:
# load rich extension
%load_ext rich

In [5]:
import glob
import json
import os
import subprocess
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from getpass import getpass
from pathlib import Path
from statistics import mean
from uuid import uuid4

import boto3
import botocore
import sagemaker
from botocore.config import Config
from rich import box, print
from rich.console import Console
from rich.progress import Progress, SpinnerColumn, TimeElapsedColumn
from rich.table import Table
from sagemaker.deserializers import JSONDeserializer
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer

from utils.autoscaling import (
    monitor_scaling_events,
    print_scaling_times,
    test_concurrency_level,
)

from utils.llmperf import (
    print_llmperf_results,
    trigger_auto_scaling,
    monitor_process,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Initiate sagemaker session

In [6]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sess.boto_region_name
config = Config(retries=dict(max_attempts=10))

boto_session = boto3.Session(region_name=region)

sagemaker_client = sess.sagemaker_client
sagemaker_runtime_client = sess.sagemaker_runtime_client
cloudwatch_client = boto3.client("cloudwatch", region_name=region, config=config)

hf_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

print(f"HF Model ID: [b green]{hf_model_id}")
print(f"Region: [b blue]{region}")
print(f"Role: [b red]{role}")

HF Model ID: meta-llama/Meta-Llama-3-8B-Instruct

Region: us-east-1

Role: arn:aws:iam::057716757052:role/gen_ai_gsmoon

## Deploy model

Create and deploy model using Amazon SageMaker HuggingFace TGI DLC

<https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy>

<div class="alert alert-block alert-warning">
<b>NOTE:</b> Remember to copy your Hugging Face Access Token from <a href="https://hf.co/">https://hf.co/</a> before running the below cell.<br/><br/>
Refer <a href="https://huggingface.co/docs/hub/security-tokens">here</a> to learn about creating HF tokens.
</div>

In [7]:
instance_type = "ml.g5.2xlarge"
suffix = f"{str(uuid4())[:5]}-{datetime.now().strftime('%d%b%Y')}"
model_name = f"Llama3-8B-fas-{suffix}"
endpoint_name = model_name
health_check_timeout = 900

HF_TOKEN = os.getenv("HUGGING_FACE_HUB_TOKEN") or getpass("Enter HUGGINGFACE Access Token: ")

# retrieve the llm image uri
# tgi_dlc = f"763104351884.dkr.ecr.{region}.amazonaws.com/huggingface-pytorch-tgi-inference:2.1-tgi2.0-gpu-py310-cu121-ubuntu22.04"
tgi_dlc = get_huggingface_llm_image_uri("huggingface", version="2.0.0")

# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "meta-llama/Meta-Llama-3-8B-Instruct",  # model_id from hf.co/models
    "SM_NUM_GPUS": "1",  # Number of GPU used per replica
    "MAX_INPUT_LENGTH": "2048",  # Max length of input text
    "MAX_TOTAL_TOKENS": "4096",  # Max length of the generation (including input text)
    "MAX_BATCH_TOTAL_TOKENS": "8192",  # Limits the number of tokens that can be processed in parallel during the generation
    "MESSAGES_API_ENABLED": "true",  # Enable the messages API
    "HUGGING_FACE_HUB_TOKEN": HF_TOKEN,
}

# create HuggingFaceModel with the image uri
print(f"Creating model: [b green]{model_name}...")
llm_model = HuggingFaceModel(name=model_name, role=role, image_uri=tgi_dlc, env=config)

# Deploy model to Amazon SageMaker endpoint
print(f"Deploying model to endpoint: [b magenta]{endpoint_name}...")
predictor = llm_model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=health_check_timeout,  # 15 minutes to be able to load the model
)

Enter HUGGINGFACE Access Token:  ········


Creating model: Llama3-8B-fas-943d7-28Jul2024...

Deploying model to endpoint: Llama3-8B-fas-943d7-28Jul2024...

-------------!

## Inference

Invoke and test endpoint using messages API. Refer to HF [Messages API](https://huggingface.co/docs/text-generation-inference/messages_api) for more info.

In [8]:
# Prompt to generate
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is deep learning?"},
]

# Generation arguments
parameters = {
    "model": hf_model_id,  # model id is required
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 512,
    "stop": ["<|eot_id|>"],
}

chat = predictor.predict({"messages": messages, **parameters})

# Unpack and print response
print(chat["choices"][0]["message"]["content"].strip())

Deep learning is a subset of machine learning that involves the use of artificial neural networks to model and 
analyze complex data. The term "deep" refers to the number of layers in the neural network, which can range from a 
few layers to hundreds or even thousands.

Traditional machine learning algorithms rely on hand-crafted features to analyze data, whereas deep learning 
algorithms can automatically learn and extract features from raw data. This is achieved through the use of multiple
layers of artificial neural networks, each of which processes and transforms the input data in a way that allows 
the network to learn increasingly complex representations of the data.

Deep learning has been incredibly successful in a wide range of applications, including:

1. Computer Vision: Deep learning has enabled machines to recognize and classify images, detect objects, and even 
generate new images.
2. Natural Language Processing: Deep learning has enabled machines to understand and generate human language, 
including speech recognition, language translation, and text summarization.
3. Speech Recognition: Deep learning has enabled machines to recognize and transcribe spoken language with high 
accuracy.
4. Robotics: Deep learning has enabled machines to learn and adapt to new situations, allowing them to perform 
complex tasks such as grasping and manipulation.
5. Game Playing: Deep learning has enabled machines to play complex games such as Go and Poker at a superhuman 
level.

The key benefits of deep learning include:

1. Ability to learn complex patterns and relationships in data
2. Ability to automatically extract features from raw data
3. Ability to generalize to new, unseen data
4. Ability to learn from large amounts of data

However, deep learning also has some challenges and limitations, including:

1. Need for large amounts of data to train the network
2. Computational requirements can be high
3. Difficulty in interpreting the decisions made by the network
4. Risk of overfitting and underfitting

Overall, deep learning has revolutionized the field of artificial intelligence and has enabled machines to perform 
tasks that were previously thought to be the exclusive domain of humans.

## Baseline average latency at various concurrency levels (Optional)

<div class="alert alert-block alert-info"><b>NOTE:</b> Running the following cell is optional<br/><br/>
By capturing average latency across various concurrency levels, we can get a fair idea on after how many concurrent request does endpoint performance would degrade significantly.<br/><br/>
Having this information can help define values for scaling policy accordingly.    
</div>

<div class="alert alert-block alert-info">
<b>INFO: ℹ️</b> Signal here is, at a given concurrency level you start to see average latency increase significantly.<br/>
At this concurrency level the endpoint gets overloaded and cannot serve requests in a timely fashion.<br/>
We use these values to set as threshold values for autoscaling.
</div>

In [9]:
# Define list of prompts
prompts = [
    "what is deep learning?",
    "what are various inference modes in Amazon SageMaker?",
    "Can I host Large language models on Amazon SageMaker?",
    "Does Amazon SageMaker support TensorRT-LLM?",
    "what is step scaling policy in the context of autoscaling ec2 instances on AWS?",
    "Why is the sky blue?",
    "List 5 benefits of incorporating limes into the diet.",
]

# Test different concurrency levels and measure average latency
concurrency_levels = [10, 50, 75, 100]  # Adjust these values as needed

for concurrency_level in concurrency_levels:
    try:
        avg_latency = test_concurrency_level(
            concurrency_level,
            prompts,
            messages,
            parameters,
            endpoint_name,
            sagemaker_runtime_client,
        )
        print(
            f"[b]Concurrency:[/b] {concurrency_level} requests,"
            f" [b]Average latency:[/b] {avg_latency:.2f} seconds"
        )
    except Exception as e:
        print(f"[b]At Concurrency[/b] {concurrency_level} requests," f"[b]Exception:[/b] \n{e}")
        continue

Concurrency: 10 requests, Average latency: 13.23 seconds

Concurrency: 50 requests, Average latency: 19.06 seconds

Concurrency: 75 requests, Average latency: 22.93 seconds

Concurrency: 100 requests, Average latency: 27.82 seconds

---

## Apply Step-Scaling autoscaling policies to endpoint

- **Step 1:** Register Scalable Target
- **Step 2:** Create Scale-Out Policy
- **Step 3:** Create Scale-In Policy
- **Step 4:** Create CloudWatch Alarms

Define and apply the step-scaling policy for scaling out.

In [10]:
variant_name = "AllTraffic"
as_min_capacity = 1
as_max_capacity = 2

resource_id = f"endpoint/{endpoint_name}/variant/{variant_name}"

autoscaling_client = boto3.client("application-autoscaling", region_name=region)

# Register scalable target
scalable_target = autoscaling_client.register_scalable_target(
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    MinCapacity=as_min_capacity,
    MaxCapacity=as_max_capacity,  # Replace with your desired maximum instances
)

scalable_target_arn = scalable_target["ScalableTargetARN"]
print(f"Resource ID: [b blue]{resource_id}")
print(f"Scalable_target_arn:\n[b green]{scalable_target_arn}")

Resource ID: endpoint/Llama3-8B-fas-943d7-28Jul2024/variant/AllTraffic

Scalable_target_arn:
arn:aws:application-autoscaling:us-east-1:057716757052:scalable-target/056ma2342ee9e13f4c46b2a2dc2b454b1edd

### Create StepScaling <span style='color:green'>Scale-out</span> Policy

In [11]:
# Configure step scaling scale-out policy
scale_out_policy_response = autoscaling_client.put_scaling_policy(
    PolicyName=f"{endpoint_name}-ScaleOutPolicy",
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    PolicyType="StepScaling",
    StepScalingPolicyConfiguration={
        "AdjustmentType": "ChangeInCapacity",
        "Cooldown": 300,  # 5 minutes cooldown
        "MetricAggregationType": "Maximum",
        "StepAdjustments": [
            {
                "MetricIntervalLowerBound": 0,
                "MetricIntervalUpperBound": 20,
                "ScalingAdjustment": 1,  # Increase by one instance
            },
            {
                "MetricIntervalLowerBound": 20,
                "ScalingAdjustment": 2,  # Increase by 2 instances
            },
        ],
    },
)

# print(scale_out_policy_response)
scale_out_policy_arn = scale_out_policy_response["PolicyARN"]
print(f"Step scaling policy ARN: [i green]{scale_out_policy_arn}[/i green]")

Step scaling policy ARN: 
arn:aws:autoscaling:us-east-1:057716757052:scalingPolicy:a2342ee9-e13f-4c46-b2a2-dc2b454b1edd:resource/sagemaker/en
dpoint/Llama3-8B-fas-943d7-28Jul2024/variant/AllTraffic:policyName/Llama3-8B-fas-943d7-28Jul2024-ScaleOutPolicy

### Create StepScaling <span style='color:green'>Scale-In</span> Policy

In [12]:
scale_in_policy_response = autoscaling_client.put_scaling_policy(
    PolicyName=f"{endpoint_name}-ScaleInPolicy",
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    PolicyType="StepScaling",
    StepScalingPolicyConfiguration={
        "AdjustmentType": "ChangeInCapacity",
        "Cooldown": 300,  # Cooldown period after scale-in activity
        "MetricAggregationType": "Maximum",
        "StepAdjustments": [
            {
                "MetricIntervalUpperBound": 0,
                "MetricIntervalLowerBound": -20,
                "ScalingAdjustment": -1,  # Decrease by 1 instance
            },
            {"MetricIntervalUpperBound": -20, "ScalingAdjustment": -2},  # Decrease by 2 instances
        ],
    },
)

# print(scale_in_policy_response)
scale_in_policy_arn = scale_in_policy_response["PolicyARN"]
print(f"Step scaling policy ARN: [i green]{scale_in_policy_arn}[/i green]")

Step scaling policy ARN: 
arn:aws:autoscaling:us-east-1:057716757052:scalingPolicy:a2342ee9-e13f-4c46-b2a2-dc2b454b1edd:resource/sagemaker/en
dpoint/Llama3-8B-fas-943d7-28Jul2024/variant/AllTraffic:policyName/Llama3-8B-fas-943d7-28Jul2024-ScaleInPolicy

### Create CloudWatch alarms (Step-Scaling)

Create CloudWatch Alarms using new <span style='color:green'><b>ConcurrentRequestsPerModel</b></span> high-resolution Metric.

In [13]:
# Define the alarm parameters for scale-out
alarm_name_scale_out = f"Step-Scaling-AlarmHigh-SageMaker:{resource_id}"
metric_name = "ConcurrentRequestsPerModel"
namespace = "AWS/SageMaker"  # CloudWatch Namespace to write metric data
statistic = "Maximum"
period = 60  # 10 seconds
evaluation_periods = 3
threshold = 20.0  # Threshold for scale-out
comparison_operator = "GreaterThanOrEqualToThreshold"
dimensions = [
    {"Name": "EndpointName", "Value": endpoint_name},
    {"Name": "VariantName", "Value": "AllTraffic"},
]
alarm_actions = [scale_out_policy_response["PolicyARN"]]
treat_missing_data = "ignore"

# create CloudWatch alarm for scale-out
response = cloudwatch_client.put_metric_alarm(
    AlarmName=alarm_name_scale_out,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic=statistic,
    Period=period,
    EvaluationPeriods=evaluation_periods,
    Threshold=threshold,
    ComparisonOperator=comparison_operator,
    Dimensions=dimensions,
    AlarmActions=alarm_actions,
    TreatMissingData=treat_missing_data,
)

print(f"CloudWatch alarm created for scale-out:\n[b blue]{alarm_name_scale_out}")

# Define the alarm parameters for scale-in
alarm_name_scale_in = f"Step-Scaling-AlarmLow-SageMaker:{resource_id}"
comparison_operator = "LessThanOrEqualToThreshold"
threshold = 10.0  # Adjust based on your requirements
alarm_actions = [scale_in_policy_response["PolicyARN"]]

# Create CloudWatch alarm for scale-in
response = cloudwatch_client.put_metric_alarm(
    AlarmName=alarm_name_scale_in,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic=statistic,
    Period=period,
    EvaluationPeriods=evaluation_periods,
    Threshold=threshold,
    ComparisonOperator=comparison_operator,
    Dimensions=dimensions,
    AlarmActions=alarm_actions,
    TreatMissingData=treat_missing_data,
)

print(f"CloudWatch alarm created for scale-in:\n[b blue]{alarm_name_scale_in}")

CloudWatch alarm created for scale-out:
Step-Scaling-AlarmHigh-SageMaker:endpoint/Llama3-8B-fas-943d7-28Jul2024/variant/AllTraffic

CloudWatch alarm created for scale-in:
Step-Scaling-AlarmLow-SageMaker:endpoint/Llama3-8B-fas-943d7-28Jul2024/variant/AllTraffic

## Trigger autoscaling action

### Use LLMPerf to generate traffic to the endpoint

Refer to <https://github.com/philschmid/llmperf> for more details on LLMPerf.

Run the LLMPerf traffic generation script in the background using `subprocess.Popen`

<div class="alert alert-block alert-info">
<b>INFO:ℹ️</b> Refer to <a href="utils/llmperf.py">utils.llmperf</a> for `trigger_autoscaling` function implementation
</div>


### Monitor Alarm Trigger times and Scaling event times
As llmperf generates traffic to the endpoint continuously this trigger auto-scaling.

The `monitor_scaling_events` function does the following:
- Calculates time taken for alarm to go into InAlarm state.
- checks if alarm is InAlarm state. If yes, then starts the scaling timer
- continuously monitors the `DesiredInstanceCount` property of the endpoint
  - waits till `CurrentInstanceCount == DesiredInstanceCount` and `EndpointStatus` is `InService`
- Calculates time taken to scale out instances prints the times in a table

The below cell triggers auto scaling action and calls the monitor_scaling_events immediately on the AlarmHigh

<div class="alert alert-block alert-info">
<b>INFO:ℹ️</b> Refer to <a href="utils/autoscaling.py">utils.autoscaling</a> for `monitor_scaling_events` function implementation
</div>

<div class="alert alert-block alert-info">
<b>NOTE: ⚠️</b>Per the <b>ScaleOut</b> Alarm, scale-out actions only start after the threshold of <b>ConcurrentRequestsPerModel >= 20</b> for 3 datapoints within <b>3 minutes</b> is breached.
</div>

In [15]:
# Trigger LLMPerf script to generate traffic to endpoint
num_concurrent_requests = 100
# LLMperf requires session credentials be passed in via environment variables.
# We'll use the current session to get these credentials.
creds = boto_session.get_credentials()
process = trigger_auto_scaling(creds, region, endpoint_name, num_concurrent_requests)
print(f"[b green]Process ID for LLMPerf: {process.pid}")

# Start monitoring scaling events
SLEEP_TIME = 5  # time to sleep
scaling_times = monitor_scaling_events(
    endpoint_name, alarm_name_scale_out, SLEEP_TIME, cloudwatch_client, sagemaker_client
)

# Print scaling times
console = Console()
table = print_scaling_times(scaling_times)
console.print(table)

Calling LLMPerf shell script: 
/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3/notebook/02-nav
er-news-llama3-inference/backup/01-Llama3-8B-Autoscaling/trigger_autoscaling.sh

Launching LLMPerf with 100 concurrent requests

Process ID for LLMPerf: 34324

Initial instance count: 1

Tracking Alarm: Step-Scaling-AlarmHigh-SageMaker:endpoint/Llama3-8B-fas-943d7-28Jul2024/variant/AllTraffic

Output()

                  Scaling Times                   
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Target Instance Count ┃ Scaling Time (seconds) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│                     2 │                 443.06 │
└───────────────────────┴────────────────────────┘

### Monitor if the background process (llmperf) is completed.

In [16]:
# Monitor the background traffic generation process for completion
monitor_process(process)

Process 34324 finished with return code 0

Process output:
Installing llmperf...
Created results directory.
Starting benchmarking scripts on endpoint Llama3-8B-fas-943d7-28Jul2024 ...
[33m(raylet)[0m WARNING: 208 PYTHON worker processes have been started on node: 
64b995eb347fec5e59979463b391dfce353cef482715c69e6b81af1d with address: 172.16.34.227. This could be a result of 
using a large number of actors, or due to tasks blocked in ray.get() calls (see 
https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
\Results for token benchmark for Llama3-8B-fas-943d7-28Jul2024 queried with the sagemaker api.

inter_token_latency_s
    p25 = 0.020475325386183834
    p50 = 0.028829880094834434
    p75 = 0.03611504707108087
    p90 = 0.04241175232478333
    p95 = 0.04829074106144375
    p99 = 0.06677857892707872
    mean = 0.02925097575645347
    min = 0.001577030753329377
    max = 0.12492993504261482
    stddev = 0.012148928824402251
ttft_s
    p25 = 4.4758880930021405
    p50 = 9.728483260958456
    p75 = 15.018789877707604
    p90 = 15.032955799181945
    p95 = 15.052059391268994
    p99 = 15.056359737468883
    mean = 9.17332076690183
    min = 0.12697537010535598
    max = 15.110806497046724
    stddev = 5.12268800563835
end_to_end_latency_s
    p25 = 14.16301876725629
    p50 = 20.16649883845821
    p75 = 24.805583854671568
    p90 = 27.442988431616687
    p95 = 28.690741551946843
    p99 = 31.441285775075666
    mean = 19.280672544163185
    min = 1.2208690058905631
    max = 34.318980051903054
    stddev = 6.767100030438725
request_output_throughput_token_per_s
    p25 = 27.689091364060328
    p50 = 34.68598527958418
    p75 = 48.83876679673854
    p90 = 66.82974991560069
    p95 = 85.83036127207903
    p99 = 136.80032409113423
    mean = 42.31630010904782
    min = 8.004436337928507
    max = 633.9746494222823
    stddev = 30.3626819534128
number_input_tokens
    p25 = 446.75
    p50 = 552.0
    p75 = 649.25
    p90 = 749.1
    p95 = 804.0
    p99 = 905.06
    mean = 549.807
    min = 127
    max = 994
    stddev = 152.46478695837368
number_output_tokens
    p25 = 577.0
    p50 = 688.0
    p75 = 794.25
    p90 = 913.1
    p95 = 975.3499999999997
    p99 = 1102.12
    mean = 688.306
    min = 177
    max = 1198
    stddev = 171.57297470653603
Number Of Errored Requests: 0
Overall Output Throughput: 1632.563490540167
Number Of Completed Requests: 1000
Completed Requests Per Minute: 142.3114275226571
[33m(raylet)[0m WARNING: 280 PYTHON worker processes have been started on node: 
64b995eb347fec5e59979463b391dfce353cef482715c69e6b81af1d with address: 172.16.34.227. This could be a result of 
using a large number of actors, or due to tasks blocked in ray.get() calls (see 
https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
Execution time was 429 secs.

Process errors:
Cloning into 'llmperf'...
[2mResolved [1m104 packages[0m in 1.15s[0m
[2mDownloaded [1m38 packages[0m in 1.54s[0m
[2mUninstalled [1m2 packages[0m in 64ms[0m
[2mInstalled [1m38 packages[0m in 67ms[0m
 [32m+[39m [1mannotated-types[0m[2m==0.7.0[0m
 [31m-[39m [1mboto3[0m[2m==1.34.142[0m
 [32m+[39m [1mboto3[0m[2m==1.34.101[0m
 [31m-[39m [1mbotocore[0m[2m==1.34.142[0m
 [32m+[39m [1mbotocore[0m[2m==1.34.101[0m
 [32m+[39m [1mcachetools[0m[2m==5.4.0[0m
 [32m+[39m [1mdistro[0m[2m==1.9.0[0m
 [32m+[39m [1mdocopt[0m[2m==0.6.2[0m
 [32m+[39m [1mdocstring-parser[0m[2m==0.16[0m
 [32m+[39m [1mgoogle-api-core[0m[2m==2.19.1[0m
 [32m+[39m [1mgoogle-auth[0m[2m==2.32.0[0m
 [32m+[39m [1mgoogle-cloud-aiplatform[0m[2m==1.60.0[0m
 [32m+[39m [1mgoogle-cloud-bigquery[0m[2m==3.25.0[0m
 [32m+[39m [1mgoogle-cloud-core[0m[2m==2.4.1[0m
 [32m+[39m [1mgoogle-cloud-resource-manager[0m[2m==1.12.4[0m
 [32m+[39m [1mgoogle-cloud-storage[0m[2m==2.18.0[0m
 [32m+[39m [1mgoogle-crc32c[0m[2m==1.5.0[0m
 [32m+[39m [1mgoogle-resumable-media[0m[2m==2.7.1[0m
 [32m+[39m [1mgoogleapis-common-protos[0m[2m==1.63.2[0m
 [32m+[39m [1mgrpc-google-iam-v1[0m[2m==0.13.1[0m
 [32m+[39m [1mgrpcio[0m[2m==1.65.1[0m
 [32m+[39m [1mgrpcio-status[0m[2m==1.62.2[0m
 [32m+[39m [1mhuggingface-hub[0m[2m==0.24.2[0m
 [32m+[39m [1mlitellm[0m[2m==1.42.5[0m
 [32m+[39m [1mllmperf[0m[2m==0.1.0 (from 
file:///home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3/notebook
/02-naver-news-llama3-inference/backup/01-Llama3-8B-Autoscaling/llmperf)[0m
 [32m+[39m [1mnum2words[0m[2m==0.5.13[0m
 [32m+[39m [1mopenai[0m[2m==1.37.1[0m
 [32m+[39m [1mproto-plus[0m[2m==1.24.0[0m
 [32m+[39m [1mpyasn1-modules[0m[2m==0.4.0[0m
 [32m+[39m [1mpydantic[0m[2m==2.4.2[0m
 [32m+[39m [1mpydantic-core[0m[2m==2.10.1[0m
 [32m+[39m [1mpython-dotenv[0m[2m==1.0.1[0m
 [32m+[39m [1mray[0m[2m==2.33.0[0m
 [32m+[39m [1msafetensors[0m[2m==0.4.3[0m
 [32m+[39m [1mshapely[0m[2m==2.0.5[0m
 [32m+[39m [1mshellingham[0m[2m==1.5.4[0m
 [32m+[39m [1mtiktoken[0m[2m==0.7.0[0m
 [32m+[39m [1mtokenizers[0m[2m==0.19.1[0m
 [32m+[39m [1mtransformers[0m[2m==4.43.3[0m
 [32m+[39m [1mtyper[0m[2m==0.12.3[0m
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, 
configuration and file/data utilities can be used.
2024-07-28 07:14:06,326 INFO worker.py:1781 -- Started a local Ray instance.
You are using the default legacy behaviour of the <class 
'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that 
the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set
`legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this 
was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer 
from a GGUF file you can ignore this message.
  0%|          | 0/1000 [00:00<?, ?it/s]  0%|          | 0/1000 [00:00<?, ?it/s]  0%|          | 0/1000 [00:00<?, 
?it/s]  0%|          | 0/1000 [00:00<?, ?it/s]  0%|          | 0/1000 [00:00<?, ?it/s]  0%|          | 0/1000 
[00:00<?, ?it/s]  0%|          | 0/1000 [00:01<?, ?it/s]  0%|          | 0/1000 [00:01<?, ?it/s]  0%|          | 
0/1000 [00:01<?, ?it/s][36m(pid=37646)[0m None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't
be available and only tokenizers, configuration and file/data utilities can be used.
  0%|          | 0/1000 [00:01<?, ?it/s][36m(SageMakerClient pid=37646)[0m You are using the default legacy 
behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, 
and simply m

## Print LLMPerf results

LLMPerf writes the results to **"results/"** directory. `summary.json` file has the endpoint benchmarking data.

In [17]:
print_llmperf_results(num_concurrent_requests)

            LLMPerf Endpoint Metrics             
                           ╷                     
                    Metric │ Units               
 ══════════════════════════╪════════════════════ 
       Concurrent requests │ 100                 
   Avg. Input token length │ 550                 
  Avg. Output token length │ 150                 
  Avg. First-Time-To-Token │ 9173.32ms           
           Avg. Thorughput │ 1632.56 tokens/sec  
              Avg. Latency │ 29.25ms/token       
                           ╵

### Monitor Scale-in action scaling times (Optional)


<div class="alert alert-block alert-info">
<b>NOTE: ⚠️</b>Per the <b>ScaleIn</b> Alarm, scale-in actions only start after the threshold of <b>ConcurrentRequestsPerModel <= 10</b> for 3 datapoints within <b>3 minutes</b> is breached.
</div>

In [18]:
# Start monitoring scaling events
SLEEP_TIME = 5  # time to sleep
scaling_times = monitor_scaling_events(
    endpoint_name,
    alarm_name_scale_in,  # scale_in cloudwatch metric alarm name
    SLEEP_TIME,
    cloudwatch_client,
    sagemaker_client,
)

# Print scaling times
console = Console()
table = print_scaling_times(scaling_times)
console.print(table)

Initial instance count: 1

Tracking Alarm: Step-Scaling-AlarmLow-SageMaker:endpoint/Llama3-8B-fas-943d7-28Jul2024/variant/AllTraffic

Output()

                  Scaling Times                   
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Target Instance Count ┃ Scaling Time (seconds) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│                     1 │                   5.09 │
└───────────────────────┴────────────────────────┘

## Cleanup

- Delete cloudwatch alarms
- Delete scaling policies
- Deregister scalable target
- Delete model
- Delete endpoint

In [19]:
# Delete CloudWatch alarms created for Step scaling policy
alarm_names = [alarm_name_scale_out, alarm_name_scale_in]

for alarm in alarm_names:
    try:
        cloudwatch_client.delete_alarms(AlarmNames=[alarm])
        print(f"Deleted CloudWatch scale-out alarm [b]{alarm} ✅")
    except cloudwatch_client.exceptions.ResourceNotFoundException:
        print(f"CloudWatch scale-out alarm [b]{alarm}[/b] not found.")


# Delete scaling policies
print("---" * 10)
step_policies = [f"{endpoint_name}-ScaleInPolicy", f"{endpoint_name}-ScaleOutPolicy"]
for policy_name in step_policies:
    try:
        autoscaling_client.delete_scaling_policy(
            PolicyName=policy_name,
            ServiceNamespace="sagemaker",
            ResourceId=resource_id,
            ScalableDimension="sagemaker:variant:DesiredInstanceCount",
        )
        print(f"Deleted scaling policy [i green]{policy_name} ✅")
    except autoscaling_client.exceptions.ObjectNotFoundException:
        print(f"Scaling policy [i]{policy_name}[/i] not found.")

# Deregister scalable target
try:
    autoscaling_client.deregister_scalable_target(
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    )
    print(f"Scalable target for [b]{resource_id}[/b] deregistered. ✅")
except autoscaling_client.exceptions.ObjectNotFoundException:
    print(f"Scalable target for [b]{resource_id}[/b] not found!.")

print("---" * 10)
# Delete model and endpoint
try:
    print(f"Deleting model: [b green]{model_name} ✅")
    predictor.delete_model()
except Exception as e:
    print(f"{e}")

try:
    print(f"Deleting endpoint: [b magenta]{predictor.endpoint_name} ✅")
    predictor.delete_endpoint()
except Exception as e:
    print(f"{e}")

print("---" * 10)
print("Done")

Deleted CloudWatch scale-out alarm 
Step-Scaling-AlarmHigh-SageMaker:endpoint/Llama3-8B-fas-943d7-28Jul2024/variant/AllTraffic ✅

Deleted CloudWatch scale-out alarm 
Step-Scaling-AlarmLow-SageMaker:endpoint/Llama3-8B-fas-943d7-28Jul2024/variant/AllTraffic ✅

------------------------------

Deleted scaling policy Llama3-8B-fas-943d7-28Jul2024-ScaleInPolicy ✅

Deleted scaling policy Llama3-8B-fas-943d7-28Jul2024-ScaleOutPolicy ✅

...

------------------------------

Deleting model: Llama3-8B-fas-943d7-28Jul2024 ✅

Deleting endpoint: Llama3-8B-fas-943d7-28Jul2024 ✅

------------------------------

Done

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/inference|generativeai|huggingfacetgi|meta-llama|llama3-8b|faster-autoscaling|realtime-endpoints|FasterAutoscaling-SME-Llama3-8B-StepScaling.ipynb)
